<a href="https://colab.research.google.com/github/JS2387/text-mining/blob/master/reddit_climate_change_MLclassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 5.2 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/the-reddit-climate-change-dataset-comments.csv"
data = pd.read_csv(path)

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/cleaned_comments.csv"
df = pd.read_csv(path)

In [11]:
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, roc_curve, auc, precision_recall_curve

In [7]:
df = data.sample(frac = 0.02, random_state = 123)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90898 entries, 1857107 to 1953982
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            90898 non-null  object 
 1   id              90898 non-null  object 
 2   subreddit.id    90898 non-null  object 
 3   subreddit.name  90898 non-null  object 
 4   subreddit.nsfw  90898 non-null  bool   
 5   created_utc     90898 non-null  int64  
 6   permalink       90898 non-null  object 
 7   body            90898 non-null  object 
 8   sentiment       90898 non-null  float64
 9   score           90898 non-null  int64  
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 7.0+ MB


In [8]:
def create_label(sentiment):
    if sentiment < 0:
        return 0 # NEGATIVE SENTIMENT
    else:
        return 1 # POSITIVE SENTIMENT

In [9]:
df['label'] = df['sentiment'].apply(create_label)
df = df[['body', 'label']]
df = df.reset_index(drop = True)
df.head()

,body,label
0,A person's beliefs and needs shift based on ci...,1
1,I guess I don't understand what that has to do...,1
2,&gt;climate change will be way worse\n\nThere ...,0
3,The fire itself may have been caused by climat...,0
4,This has been a really interesting and informa...,1


In [10]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = roc_curve(target, predictions)
    roc_auc = auc(fpr, tpr)
    return roc_auc

In [12]:
df.label.value_counts()

1    47928
0    42970
Name: label, dtype: int64

In [13]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df.body)
X.shape

(90898, 120674)

In [14]:
Y = df.label

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5, random_state = 42, stratify = Y)

In [16]:
# Classifiers

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# Dummy CLassifier
#DC = DummyClassifier(strategy = 'stratified') 

# Naive Bayes
clf_NB = MultinomialNB()

# Logisitic Regression
clf_LogR = LogisticRegression(max_iter=1000)

# Logistic Regression with CV
#clf_LogRCV = LogisticRegressionCV(max_iter = 1000)

# SGD
clf_sgd = SGDClassifier(tol=1e-3)

# SVC
#clf_svc = SVC(decision_function_shape = 'ovo')

# Random Forest
clf_RF = RandomForestClassifier(random_state=42, n_estimators=100)

# Adaptive Boosting
clf_AdaB = AdaBoostClassifier(random_state=42, n_estimators=100, learning_rate=0.9)

# xGBoost
clf_xGB = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1)

# Ensemble classifier
#ensemble_clf = VotingClassifier(estimators=[
#    ('RF_clf', clf_RF),
#    ('AdaB_clf', clf_AdaB),
#    ('xGB_clf', clf_xGB)
#    ], voting='soft')

In [17]:
# Naive bayes
clf_NB.fit(X_train, Y_train)
pred_NB = clf_NB.predict(X_test)
print(classification_report(Y_test, pred_NB))

              precision    recall  f1-score   support

           0       0.64      0.78      0.71     21485
           1       0.76      0.61      0.68     23964

    accuracy                           0.69     45449
   macro avg       0.70      0.70      0.69     45449
weighted avg       0.70      0.69      0.69     45449



In [18]:
# Logisitic Regression
clf_LogR.fit(X_train, Y_train)
pred_LogR = clf_LogR.predict(X_test)
print(classification_report(Y_test, pred_LogR))

              precision    recall  f1-score   support

           0       0.81      0.78      0.79     21485
           1       0.81      0.84      0.82     23964

    accuracy                           0.81     45449
   macro avg       0.81      0.81      0.81     45449
weighted avg       0.81      0.81      0.81     45449



In [25]:
roc_auc(pred_LogR, Y_test)

0.8075416203973946

In [19]:
# Stochastic Gradient Descent
clf_sgd.fit(X_train, Y_train)
pred_sgd = clf_sgd.predict(X_test)
print(classification_report(Y_test, pred_sgd))

              precision    recall  f1-score   support

           0       0.85      0.71      0.77     21485
           1       0.77      0.89      0.83     23964

    accuracy                           0.80     45449
   macro avg       0.81      0.80      0.80     45449
weighted avg       0.81      0.80      0.80     45449



In [26]:
roc_auc(pred_sgd, Y_test)

0.7978364131023158

In [23]:
# SGD with Grid Search CV

from sklearn.model_selection import GridSearchCV

sgdCV_parameters_grid = {
    'max_iter': (20,),
    'alpha': (0.00001, 0.000001),
    'penalty': ('l2', 'elasticnet'),
}

# Model and fit with grid search
gs_sgd_clf = GridSearchCV(clf_sgd, sgdCV_parameters_grid, cv=5, n_jobs=-1, verbose=1)
gs_sgd_clf.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


GridSearchCV(cv=5, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'alpha': (1e-05, 1e-06), 'max_iter': (20,),
                         'penalty': ('l2', 'elasticnet')},
             verbose=1)

In [24]:
# results of the best model along with the parameter values
print(f'The best score: {gs_sgd_clf.best_score_}')
print(f'Alpha val:', gs_sgd_clf.best_params_['alpha'])

# Prediction
gs_sgd_clf_pred = gs_sgd_clf.predict(X_test)

# Classification report
class_report_sgd = classification_report(Y_test, gs_sgd_clf_pred)
print(class_report_sgd)

The best score: 0.7854078159493801
Alpha val: 1e-05
              precision    recall  f1-score   support

           0       0.79      0.76      0.77     21485
           1       0.79      0.82      0.81     23964

    accuracy                           0.79     45449
   macro avg       0.79      0.79      0.79     45449
weighted avg       0.79      0.79      0.79     45449



In [30]:
roc_auc(gs_sgd_clf_pred, Y_test)

0.7898588603174718

In [20]:
# Random Forest
clf_RF.fit(X_train, Y_train)
pred_RF = clf_RF.predict(X_test)
print(classification_report(Y_test, pred_RF))

              precision    recall  f1-score   support

           0       0.75      0.67      0.71     21485
           1       0.73      0.81      0.77     23964

    accuracy                           0.74     45449
   macro avg       0.74      0.74      0.74     45449
weighted avg       0.74      0.74      0.74     45449



In [27]:
roc_auc(pred_RF, Y_test)

0.7366926446608862

In [21]:
# Adaptive Boosting
clf_AdaB.fit(X_train, Y_train)
pred_AdaB = clf_AdaB.predict(X_test)
print(classification_report(Y_test, pred_AdaB))

              precision    recall  f1-score   support

           0       0.77      0.63      0.69     21485
           1       0.71      0.83      0.77     23964

    accuracy                           0.73     45449
   macro avg       0.74      0.73      0.73     45449
weighted avg       0.74      0.73      0.73     45449



In [28]:
roc_auc(pred_AdaB, Y_test)

0.7280587324629796

In [22]:
# xGBoost
clf_xGB.fit(X_train, Y_train)
pred_xGB = clf_xGB.predict(X_test)
print(classification_report(Y_test, pred_xGB))

              precision    recall  f1-score   support

           0       0.76      0.65      0.70     21485
           1       0.72      0.82      0.77     23964

    accuracy                           0.74     45449
   macro avg       0.74      0.74      0.74     45449
weighted avg       0.74      0.74      0.74     45449



In [29]:
roc_auc(pred_xGB, Y_test)

0.7353059639494149